In [1]:
import os

In [5]:
%pwd

'c:\\Users\\ubaid\\Downloads\\MLOps\\SalesForecasting\\research'

In [6]:
os.chdir('../')

In [7]:
pwd

'c:\\Users\\ubaid\\Downloads\\MLOps\\SalesForecasting'

In [8]:
import os
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import yaml
from dataclasses import dataclass
from pathlib import Path
import ipaddress
from videogamesforecasting.constants import *
from videogamesforecasting.utils.common import read_yaml, create_directories



# included it in src\videogamesforecasting\entity\config_entity.py
@dataclass(frozen=True)
class DataIngestionConfig:
    s_type: str
    host: ipaddress.IPv4Address
    port: int
    database: str
    username: str
    password: str
    query: str
    d_type: str
    path: Path
    include_column_names: str

In [ ]:
# included it in src\videogamesforecasting\config\configuration.py
class ConfigurationManager:
    def __init__(
        self,
        # from constants
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        (create_directories([self.config.artifacts_root]))

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([os.path.split(config.destination.path)[0]])

        data_ingestion_config = DataIngestionConfig(
            s_type= config.source.type,
            host= config.source.host,
            port= config.source.port,
            database= config.source.database,
            username= config.source.username,
            password= config.source.password,
            query= config.source.query,
            d_type=config.destination.type,
            path= config.destination.path,
            include_column_names= config.destination.include_column_names
        )

        return data_ingestion_config

In [ ]:
# included it in src\videogamesforecasting\components\data_ingestion.py

from videogamesforecasting.logging import logger
class DataIngestion:
    def __init__(self, config_path="config/config.yaml"):
        with open(config_path, "r") as config_file:
            self.config = yaml.safe_load(config_file)["data_ingestion"]

        self.source_config = self.config["source"]
        self.destination_config = self.config["destination"]

    def connect_to_mysql(self):
        engine = create_engine(
            f"mysql+pymysql://{self.source_config['username']}:{self.source_config['password']}@{self.source_config['host']}:{self.source_config['port']}/{self.source_config['database']}"
        )
        return engine.connect()

    def fetch_data_from_mysql(self):
        connection = self.connect_to_mysql()
        query = self.source_config["query"]
        data = pd.read_sql(query, connection)
        connection.close()
        return data

    def save_to_csv(self, data):
        path = self.destination_config["path"]
        include_column_names = self.destination_config.get("include_column_names", False)

        data.to_csv(path, index=False, header=include_column_names)

    def run_ingestion(self):
        data = self.fetch_data_from_mysql()
        self.save_to_csv(data)
        path = self.destination_config["path"]
        file=os.path.split(path)[1]

        logger.info(f"{file} is saved")
        



In [ ]:
try:
    config_manager=ConfigurationManager()
    config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion()
    data_ingestion.run_ingestion()
except Exception as e:
    raise e